<a href="https://colab.research.google.com/github/kcarroth-dev/KarltonCarrothers_ML/blob/main/ECGR4105_Hw7_1a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time
from sklearn.metrics import f1_score, confusion_matrix

In [ ]:
# Load CIFAR-10 dataset to calculate mean and std
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())

# Calculate mean and std
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transformation with calculated mean and std
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# Load CIFAR-10 dataset with normalization
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.tanh(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model
model = CNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Training loop
num_epochs = 300
total_start_time = time.time()
for epoch in range(num_epochs):
    start_time = time.time()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    end_time = time.time()
    training_time = end_time - start_time

    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Training Time: {training_time:.2f} seconds')

total_end_time = time.time()
total_training_time = total_end_time - total_start_time
print(f'Total Training Time: {total_training_time:.2f} seconds')

# Testing the model
model.eval()
correct = 0
total = 0
all_predicted = []
all_labels = []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        all_predicted.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

# Set the model back to training mode
model.train()

# Calculate and print F1 score at the end
f1 = f1_score(all_labels, all_predicted, average='weighted')
print(f'Final F1 Score: {f1:.4f}')

# Calculate and print confusion matrix at the end
cm = confusion_matrix(all_labels, all_predicted)
print('Final Confusion Matrix:')
print(cm)

# After training is complete, print final evaluation accuracy
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')

100%|██████████| 170M/170M [00:02<00:00, 64.2MB/s]


Epoch 1/300, Loss: 2.236559885542106, Training Time: 103.57 seconds
Epoch 11/300, Loss: 1.5235683155791533, Training Time: 99.57 seconds
Epoch 21/300, Loss: 1.2897768347617, Training Time: 98.32 seconds
Epoch 31/300, Loss: 1.1486559276995452, Training Time: 98.46 seconds
Epoch 41/300, Loss: 1.0379858782224338, Training Time: 98.52 seconds
Epoch 51/300, Loss: 0.9446637147985151, Training Time: 98.69 seconds
Epoch 61/300, Loss: 0.8620900610066435, Training Time: 99.81 seconds
Epoch 71/300, Loss: 0.7833958034930022, Training Time: 98.23 seconds
Epoch 81/300, Loss: 0.7047612826766261, Training Time: 101.23 seconds
Epoch 91/300, Loss: 0.6268772250779754, Training Time: 99.08 seconds
Epoch 101/300, Loss: 0.546857426462271, Training Time: 97.87 seconds
Epoch 111/300, Loss: 0.4673825012677161, Training Time: 99.09 seconds
Epoch 121/300, Loss: 0.3911506297719448, Training Time: 98.72 seconds
Epoch 131/300, Loss: 0.3197563577948324, Training Time: 98.01 seconds
Epoch 141/300, Loss: 0.25574210005